In [5]:
import json
import pprint
import subprocess

import duckdb
import polars as pl

In [6]:
import os
import sys
import django

# Add your project root to the Python path if necessary
sys.path.append('/home/reepoi/GitHub/Orange-Button-ChecklistRegistry') 

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
# Set the DJANGO_SETTINGS_MODULE environment variable
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'checklist.settings')

# Initialize Django
django.setup()

In [7]:
from server import models

### Maintainer

In [8]:
maintainer, was_created = models.ChecklistTemplateMaintainer.objects.update_or_create(
    URL_Value='https://ecogyenergy.com',
    WorkPhone_Value='(718) 304-0945',
    ChecklistTemplateMaintainerName_Value='Ecogy Energy',
)
f'{maintainer=}, {was_created=}'
# maintainer.save()

'maintainer=<ChecklistTemplateMaintainer: ChecklistTemplateMaintainer object (2)>, was_created=False'

### Upsert AnswerOptions
The AnswerOption table should have a unique constaint on Value, and the columns of this table are id and Value. Value can be treated as an alternate key.

In [9]:
answer_option_groups = {
    'Completed?': ['', 'Completed']
}
pprint.pp(answer_option_groups, compact=True)

{'Completed?': ['', 'Completed']}


In [10]:
for k, v in answer_option_groups.items():
    answer_option_groups[k] = [
        models.AnswerOption.objects.get_or_create(Value=vv)[0] for vv in v
    ]
pprint.pp(answer_option_groups, compact=True)

{'Completed?': [<AnswerOption: >, <AnswerOption: Completed>]}


### Checklist Templates

In [11]:
checklist_template_questions = pl.read_csv('ChecklistTemplateQuestions.2026-02-11.csv')
checklist_template_questions

ChecklistTemplateName_Value,SectionName_Value,QuestionLabel_Value
str,str,str
"""Prior to Construction NTP""","""Interconnection""","""Executed ISA"""
"""Prior to Construction NTP""","""Interconnection""","""25% Interconnection Proof of P…"
"""Prior to Construction NTP""","""Interconnection""","""Wireless Meter Signal Test and…"
"""Prior to Construction NTP""","""Interconnection""","""Interconnection Complexities"""
"""Prior to Construction NTP""","""Interconnection""","""Other"""
…,…,…
"""Prior to Final PTO/COD""","""Incentive Claim/Revenue""","""Revenue Sorce in ProForma"""
"""Prior to Final PTO/COD""","""Final Completion""","""Interconnection Cost Reconcili…"
"""Prior to Final PTO/COD""","""Final Completion""","""Execute Internal Closeout Docu…"


In [13]:
checklist_templates = []
for name in checklist_template_questions['ChecklistTemplateName_Value'].unique():
    checklist_template, was_created = models.ChecklistTemplate.objects.update_or_create(
        ChecklistTemplateName_Value=name,
        ChecklistTemplateVersion_Value='1',
        ChecklistTemplateMaintainer=maintainer,
    )
    print(f'{checklist_template=}, {was_created=}')
    checklist_templates.append(checklist_template)

checklist_template=<ChecklistTemplate: ChecklistTemplate object (3)>, was_created=True
checklist_template=<ChecklistTemplate: ChecklistTemplate object (4)>, was_created=True


##### Questions

In [16]:
for template in checklist_templates:
    questions = checklist_template_questions.filter(pl.col('ChecklistTemplateName_Value') == template.ChecklistTemplateName_Value)
    question_rows = []
    for i, row in enumerate(questions.iter_rows(named=True), start=1):
        question = models.Question(
            AnswerType_Value='EnumeratedSingle',
            DisplaySeqNumber_Value=i * 10,
            QuestionLabel_Value=row['QuestionLabel_Value'],
            SectionName_Value=row['SectionName_Value'],
        )
        question.save()
        question.AnswerOptions.set(answer_option_groups['Completed?'])
        question_rows.append(question)
    template.Questions.set(question_rows)

##### Tags
The Tags table should have a unique constaint on Value, and the columns of this table are id and Value. Value can be treated as an alternate key.

In [10]:
# for template in checklist_templates:
#     tags = checklist_template_questions.filter(pl.col('Checklist Template Name') == template.ChecklistTemplateName_Value)['Tag'].unique()
#     tags = [models.Tag.objects.get_or_create(Value=tag)[0] for tag in tags]
#     template.Tags.set(tags)